In [3]:
import os

In [4]:
%pwd

'c:\\MLOPS\\MLOPS-Text-Summarizer-Project-NLP\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\MLOPS\\MLOPS-Text-Summarizer-Project-NLP'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
import sys
import os

sys.path.append(os.path.abspath(".."))
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories




In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-01-17 13:11:02,002: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-17 13:11:02,004: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-17 13:11:02,005: INFO: common: created directory at: artifacts]
[2026-01-17 13:11:02,005: INFO: common: created directory at: artifacts/data_ingestion]
[2026-01-17 13:11:02,802: INFO: 1434958058: artifacts/data_ingestion/data.zip download! with following info: 
Date: Sat, 17 Jan 2026 07:41:02 GMT
Content-Type: text/html; charset=utf-8
Vary: X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, X-Requested-With,Accept-Encoding, Accept, X-Requested-With
x-repository-download: git clone https://github.com/Shaurya25071997/MLOPS-Text-Summarizer-Project-NLP.git
x-raw-download: https://raw.githubusercontent.com/Shaurya25071997/MLOPS-Text-Summarizer-Project-NLP/main/summarizer-data.zip
ETag: W/"54349f381ef6d4f5fe774aa3ff741256"
Cache-Control: max-age=0, private, must-revalidate
Strict-Transport-Security: max-age=31

BadZipFile: File is not a zip file